# Task: Raggruppamento degli utenti in base alle loro preferenze o comportamento sulla piattaforma;

In [3]:
import pandas as pd

/var/folders/03/smk9v43n5dz1mcwcgdj4fzz40000gn/T/ipykernel_7516/714551233.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user = pd.read_csv('./dataset/user_dropped.csv')


In [ ]:
df_user = pd.read_csv('./dataset/user_dropped.csv')

In [ ]:
# user_id // string, 22 character unique user id, maps to the user in user.json
# name // string, the user's first name
# review_count // integer, the number of reviews they've written
# yelping_since // string, when the user joined Yelp, formatted like YYYY-MM-DD
# friends // array of strings, an array of the user's friend as user_ids
# useful // array of strings, an array of the user's friend as user_ids
# funny // integer, number of useful votes sent by the user
# cool // integer, number of funny votes sent by the user
# fans // integer, number of cool votes sent by the user
# elite // array of integers, the years the user was elite
# average_stars 31 // integer, number of fans the user has
# compliment_hot // float, average rating of all reviews
# compliment_more // integer, number of hot compliments received by the user
# compliment_profile // integer, number of more compliments received by the user
# compliment_cute // integer, number of profile compliments received by the user
# compliment_list // integer, number of cute compliments received by the user
# compliment_note // integer, number of list compliments received by the user
# compliment_plain // integer, number of note compliments received by the user
# compliment_cool // integer, number of plain compliments received by the user
# compliment_funny // integer, number of cool compliments received by the user
# compliment_writer // integer, number of funny compliments received by the user
# compliment_photos // integer, number of writer compliments received by the user

In [6]:
df_user.head(10)

,Unnamed: 0,index,user_id,name,review_count,yelping_since,useful,funny,cool,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,0,1887897,qTMpKPwjtSDJMD-0sdd_Ag,Russ,2,2013-01-31 00:30:02,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,1887898,yy_AsCP84HLEa4f69QcEMg,Ashleigh,1,2019-01-07 02:39:12,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,1887899,-sjaSX7O8Xq6FV_djSWtag,Joana,18,2013-11-25 14:27:36,11,6,7,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,1887900,BR6WprijmGNbAI6CAaGgYQ,diane,3,2011-09-26 03:14:14,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,1887901,SNhQvkBwMHJKSIjzV8-73w,Vanessa,1,2017-11-15 15:35:36,2,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
5,5,1887902,HiPUJrclH4PibcIqwoakww,Gabrielle,16,2016-06-12 00:08:20,5,1,2,NaN,...,0,0,0,0,0,1,0,0,0,0
6,6,1887903,2vDN6H1BHXWZIo6aJCSETA,Ann,2,2011-11-16 23:06:47,0,0,0,NaN,...,0,0,0,0,1,0,0,0,0,0
7,7,1887904,n88a8mWrSpVS55pTSjNT-A,Sharie,1,2018-08-19 13:31:18,1,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
8,8,1887905,UKiJxDwci6UctxjSPr9gDA,Sarah,1,2017-01-30 03:16:49,0,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
9,9,1887906,kyMIAJ9_DOjJhUUUqW2o-g,Suzanne,2,2011-09-19 14:57:04,1,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
